In [2]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
import datetime
import time
import json
from google.colab import files
from tweepy import Cursor
!pip install xlsxwriter  
!pip3 install twint 
import xlsxwriter
import pandas as pd
import csv
import datetime
from tweepy.streaming import StreamListener

     |████████████████████████████████| 153kB 6.5MB/s 
     |████████████████████████████████| 1.3MB 8.9MB/s 
     |████████████████████████████████| 266kB 37.8MB/s 
     |████████████████████████████████| 327kB 41.7MB/s 
     |████████████████████████████████| 143kB 31.3MB/s 
     |████████████████████████████████| 296kB 41.7MB/s 
     |████████████████████████████████| 235kB 40.8MB/s 
  Created wheel for twint: filename=twint-2.1.20-cp37-none-any.whl size=33922 sha256=fea9a40a295cfc4b27f0a9c300b311eca8c9f028bea4f97510b70670bb676ca9
  Stored in directory: /root/.cache/pip/wheels/66/25/c7/855c1d896087ef84df6e6713b8adb073aff99af119450e1e4f
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13485 sha256=b07376255a91a41113a93a8c6be1fee09d4ce48deef7a992ff59cf262652fefc
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for googletransx: filename=googletransx-2.4.2-cp37-none-any.wh

In [3]:
# load/read from file using JSON
secrets = json.loads(open("secrets_Xiaomi.json","r").read())

api_key = secrets['CONSUMER_KEY']
api_key_secret = secrets['CONSUMER_SECRET']
access_key = secrets['ACCESS_TOKEN_KEY']
access_key_secret = secrets['ACCESS_TOKEN_SECRET']


auth = OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_key, access_key_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

#change per brand
screen_name = "XiaomiSupport"

In [4]:
def getAudienceProfile():
  
  with open("XiaomiProfile.json", 'w') as f:
    result = api.get_user("XiaomiSupport")
    f.write(json.dumps(result._json, indent=3))

getAudienceProfile()

In [ ]:
def getFollowers():
  counter = 1
  with open("followers_XiaomiSupport.json","w") as fw:
    for pages in Cursor(api.followers, screen_name="XiaomiSupport", count =1000).pages(500):
      print("Retrieving 1k follower ")
      time.sleep(2)
      for follower in pages:
        fw.write(json.dumps(follower._json) + "\n")


getFollowers()

In [5]:
def dumpTimeline():
  username = "XiaomiSupport"
  fname="user_timeline_{}.json".format(username)
  
  print("Batch collection of timeline statuses for ", username)

  counter=0
  with open(fname,'w') as f:
    for page in Cursor(api.user_timeline, screen_name=username,count=50).pages(5):
      for status in page:
        f.write(json.dumps(status._json)+"\n")
        counter+=1
  print("Total %d User Timeline Tweets collected" % counter)

dumpTimeline()

Batch collection of timeline statuses for  XiaomiSupport
Total 250 User Timeline Tweets collected


In [6]:
screen_name = "XiaomiSupport"

followers_file = 'followers_XiaomiSupport.json'.format(screen_name)

with open(followers_file) as f:

    reach = []

    for line in f:

      profile = json.loads(line)

      reach.append((profile['screen_name'], profile['followers_count']))

In [7]:
profile_file = 'XiaomiProfile.json'.format(screen_name)

with open(profile_file) as f:

    profile = json.load(f)

    followers = profile['followers_count']

    tweets = profile['statuses_count']

sum_reach = sum([x[1] for x in reach])

avg_followers = round(sum_reach / followers, 2)

In [8]:
timeline_file = 'user_timeline_XiaomiSupport.json'.format(screen_name)

with open(timeline_file) as f:

    favorite_count, retweet_count = [], []

    for line in f:

        tweet = json.loads(line)

        favorite_count.append(tweet['favorite_count'])

        retweet_count.append(tweet['retweet_count'])

In [9]:
avg_favorite = round(sum(favorite_count) / tweets, 5)

avg_retweet = round(sum(retweet_count) / tweets, 5)

favorite_per_user = round(sum(favorite_count) / followers, 5)

retweet_per_user = round(sum(retweet_count) / followers, 5)

In [10]:
print("{} followers".format(followers))

print("{} users reached by 1-degree connections".format(sum_reach))

print("Average number of {}'s followers: {}".format(screen_name, avg_followers))

print("Favorited {} times ({} per tweet, {} per user)".format(sum(favorite_count), avg_favorite, favorite_per_user))

print("Retweeted {} times ({} per tweet, {} per user)".format(sum(retweet_count), avg_retweet, retweet_per_user))

4718 followers
218744 users reached by 1-degree connections
Average number of XiaomiSupport's followers: 46.36
Favorited 2478 times (5.63182 per tweet, 0.52522 per user)
Retweeted 372 times (0.84545 per tweet, 0.07885 per user)
